In [1]:
import numpy as np
import pandas as pd
from scipy.io import wavfile
import joblib
import seaborn as sns
%matplotlib tk
#%matplotlib qt
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pylab import specgram
from scipy.signal import argrelmax
from scipy.stats import linregress, pearsonr
import os

#%pylab inline
import cv2


Autosaving every 180 seconds


In [2]:

def read_filter_data(fname, call_onset, call_offset, filter_data, plotting):

    # load data
    print ("Loading: ", fname)
    data = np.load(fname)
    mic0_audio = data[0]
    mic1_audio = data[1]
    mic2_audio = data[2]
    mic3_audio = data[3]
    
    # clip data 
    #print ("Clipping :", call_onset, call_offset)
    fs = 250000
    call_mic0 = mic0_audio[int(call_onset*fs):int(call_offset*fs)]
    call_mic1 = mic1_audio[int(call_onset*fs):int(call_offset*fs)]
    call_mic2 = mic2_audio[int(call_onset*fs):int(call_offset*fs)]
    call_mic3 = mic3_audio[int(call_onset*fs):int(call_offset*fs)]  


    nfft = 1024
    if plotting:
        fig=plt.figure()
        ax1=plt.subplot(4,2,1)
        pxx, freqs, bins, im = specgram(call_mic0, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)
        plt.ylabel('mic 0')

        ax1=plt.subplot(4,2,3)
        pxx, freqs, bins, im = specgram(call_mic1, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)
        plt.ylabel('mic 1')

        ax1=plt.subplot(4,2,5)
        pxx, freqs, bins, im = specgram(call_mic2, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)
        plt.ylabel('mic 2')

        ax1=plt.subplot(4,2,7)
        pxx, freqs, bins, im = specgram(call_mic3, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)
        plt.ylabel('mic 3')

    # filter data
    start_fs = 6500
    end_fs = 7500
    mic0_audio_bp = butter_bandpass_filter(call_mic0, start_fs, end_fs, fs)  # YOU CAN AUTODETECT THIS
    mic1_audio_bp = butter_bandpass_filter(call_mic1, start_fs, end_fs, fs)
    mic2_audio_bp = butter_bandpass_filter(call_mic2, start_fs, end_fs, fs)
    mic3_audio_bp = butter_bandpass_filter(call_mic3, start_fs, end_fs, fs)

    if plotting:
        ax2=plt.subplot(4,2,2)
        pxx, freqs, bins, im = specgram(mic0_audio_bp, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)

        ax2=plt.subplot(4,2,4)
        pxx, freqs, bins, im = specgram(mic1_audio_bp, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)

        ax2=plt.subplot(4,2,6)
        pxx, freqs, bins, im = specgram(mic2_audio_bp, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)

        ax2=plt.subplot(4,2,8)
        pxx, freqs, bins, im = specgram(mic3_audio_bp, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)

        plt.suptitle(os.path.split(fname)[1])
    
    if filter_data:
        return (mic0_audio_bp, mic1_audio_bp, mic2_audio_bp, mic3_audio_bp)
    else:
        return (call_mic0, call_mic1, call_mic2, call_mic3)
    
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y



In [23]:
# Load video file
fname = "/media/cat/4TBSSD/dan/july_19/2020_07_24_15_11_26_425064/video/2020_07_24_15_11_26_425064.avi"

fname_light = fname[:-4]+"_light.npy"
if os.path.exists(fname_light)==False:
    cap = cv2.VideoCapture(fname)

    # make a downsampled version of the video; Even more should be ok
    vid = np.zeros((30000,1024//4,1280//4),'uint8')

    ctr=0
    while(cap.isOpened()):
        if ctr%1000==0:
            print (ctr)
        ret, frame = cap.read()
        vid[ctr]=frame[::4,::4].max()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        ctr+=1

    cap.release()
    cv2.destroyAllWindows()
    print ("Video file: ", vid.shape)
    # Find the pixel with LED light and track it's max over all 3 color channels over time
    light = vid[:,147, 306].max(1)
    print(light.shape)
    np.save(fname_light, light)
else:
    light = np.load(fname_light)
print ("LED Light: ", light.shape)

LED Light:  (30000,)


In [3]:
# Load audio file
mic4 = np.load('/media/cat/4TBSSD/dan/july_19/2020_07_24_15_11_26_425064/audio/2020_07_24_15_11_26_425064_audio_mic_4.npy')
print ("audio file: ", mic4.shape)

(30000,)


In [24]:
# Visualize data against each other
t_led = np.arange(light.shape[0])/25.
temp = light-light.mean(0)
plt.plot(t_led, temp/50., c='red')

# plot mic ttl pulse
mic4_subsampled = mic4[::10]
t_mic = np.arange(mic4_subsampled.shape[0])/12500.
temp = mic4_subsampled-mic4_subsampled.mean(0)
plt.plot(t_mic, temp+2.8,c='blue')

# formating
plt.title("LED: red;   TTL Pulse: Blue", fontsize=20)
plt.xlim(0,1200)
plt.yticks([])
plt.xlabel("Time (sec)",fontsize=20)
plt.show()